In [32]:
import pandas as pd
#Complete dataSet
dataSet = pd.read_csv('bank-additional-full.csv', delimiter=";")
df = pd.DataFrame(dataSet)
df.head(n=5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [33]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= sklearn.model_selection.train_test_split(dataSet, df.loc[:,"y"],test_size=.1,train_size=.9, random_state=(2021-10-25))

In [34]:
y = pd.DataFrame(dataSet, columns=["y"])
features = df.drop(columns=["duration", "y"])
X_train, X_test, y_train, y_test= sklearn.model_selection.train_test_split(features, y,test_size=.1,train_size=.9, random_state=(2021-10-25))

In [35]:
bank_client_data_train = pd.DataFrame(X_train, columns=["age","job","marital","education","default","housing","loan"])
bank_client_data_test = pd.DataFrame(X_test, columns=["age","job","marital","education","default","housing","loan"])
dummy_bcd_train = pd.get_dummies(bank_client_data_train,drop_first=True)
dummy_bcd_test = pd.get_dummies(bank_client_data_test,drop_first=True)

In [36]:
from sklearn.naive_bayes import CategoricalNB
cnb = CategoricalNB()
cnb.fit(dummy_bcd_train,y_train)
print(cnb.score(dummy_bcd_test,y_test))
print(cnb.score(dummy_bcd_train,y_train))

0.8788540907987376
0.8818689470986538


C:\Users\nikki\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [37]:
#Taking a look at the data used in the previous experiment (expirement 5)
print(dummy_bcd_train)

       age  job_blue-collar  job_entrepreneur  job_housemaid  job_management  \
22975   56                0                 0              0               0   
14746   26                0                 0              0               0   
12505   38                0                 0              0               0   
6801    33                0                 0              0               0   
18389   29                0                 0              0               0   
...    ...              ...               ...            ...             ...   
17987   31                1                 0              0               0   
20299   29                0                 0              0               0   
14556   49                1                 0              0               0   
28981   47                0                 0              0               0   
32764   35                0                 0              0               0   

       job_retired  job_self-employed  

Experiment Number 6:
Even though Categorical Naive Bayes assumes that each value of the age variable is a separate category, in our data used in the previous expeirment, there is only one category for the age variable. This is not reasonable when using the Categorical Naive Bayes classifier, since it is assuming that each age is a serparete column.

In [38]:
bins = [0,10,20,30,40,50,60,70,80,90,100]
labels = ['0-10','11-20','21-30','31-40','41-50','51-60','61-70','71-80','81-90','91-100']
bcd_train_ageSplit = bank_client_data_train
bcd_train_ageSplit['Age_Category'] = pd.cut(bank_client_data_train['age'], bins= bins, labels= labels)
bcd_train_ageSplit = bcd_train_ageSplit.drop(columns=['age'])
dummy_bcd_train_ageSplit = pd.get_dummies(bcd_train_ageSplit,drop_first=True)

bcd_test_ageSplit = bank_client_data_test
bcd_test_ageSplit['Age_Category'] = pd.cut(bank_client_data_test['age'], bins= bins, labels= labels)
bcd_test_ageSplit = bcd_test_ageSplit.drop(columns=['age'])
dummy_bcd_test_ageSplit = pd.get_dummies(bcd_test_ageSplit,drop_first=True)

cnb.fit(dummy_bcd_train_ageSplit,y_train)
print(cnb.score(dummy_bcd_test_ageSplit,y_test))
print(cnb.score(dummy_bcd_train_ageSplit,y_train))

0.8788540907987376
0.881437319593191


C:\Users\nikki\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [39]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
knn.fit(dummy_bcd_train,y_train)
##print(knn.score(dummy_bcd_test,y_test))
##print(knn.score(dummy_bcd_train,y_train))

C:\Users\nikki\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [40]:
y_test['y'].value_counts()

no     3646
yes     473
Name: y, dtype: int64

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
dumb_class = np.zeros_like(y_test, dtype='uint')
y_test_dummy = pd.get_dummies(y_test, drop_first =True)
conf_matrix = confusion_matrix(y_test_dummy.values,dumb_class)
print("Confusion Matrix:\n", conf_matrix)

score_auc = roc_auc_score(y_test_dummy.values,dumb_class)
print("AUC Score:\n", score_auc )

Confusion Matrix:
 [[3646    0]
 [ 473    0]]
AUC Score:
 0.5


In [52]:
pred_bins = cnb.predict(dummy_bcd_test_ageSplit)
score_bins = cnb.score(dummy_bcd_test_ageSplit,y_test)
print("Score:", score_bins)
conf_matrix_bins = confusion_matrix(pred_bins, y_test)
print("Confusion Matrix:\n", conf_matrix_bins)

predictProb_y = cnb.predict_proba(dummy_bcd_test_ageSplit)[:,1]
fps, tps, thresholds = roc_curve(y_test, predictProb_y,pos_label=1)
roc_score = roc_auc_score(y_test, predictProb_y)
print("CategoricalNB - ROC AUC score", roc_score)

pred_KNN = knn.predict(dummy_bcd_test)
score_KNN = knn.score(dummy_bcd_test)
conf_matrix_KNN = 

Score: 0.8788540907987376
Confusion Matrix:
 [[3554  407]
 [  92   66]]
CategoricalNB - ROC AUC score 0.6389724207594062


C:\Users\nikki\anaconda3\lib\site-packages\sklearn\metrics\_ranking.py:949: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn("No positive samples in y_true, "
